In [1]:
import codecs
import json
import lmdb
import logging
import math
import multiprocessing as mp
import nltk
import numpy as np
import os
import pickle
import random
import re
import scipy.sparse as sp
import tensorflow as tf
import time
import torch

from collections import defaultdict as dd
from datetime import datetime
from functools import lru_cache
from gensim.models import Word2Vec
from itertools import chain
from numpy.random import shuffle
from os.path import join
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'  # 0 = All logs are displayed, including debug information
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda-12.4/nvvm/libdevice'
tf.config.optimizer.set_jit(False) 
tf.config.set_visible_devices([], 'GPU')
# Verify that the configuration is correct
print("Visible devices:", tf.config.get_visible_devices())

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, model_from_json, Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, Layer, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import register_keras_serializable
from transformers import BertModel, BertTokenizer
from __future__ import division
from __future__ import print_function

2024-05-02 15:12:55.808409: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 15:12:56.282888: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-02 15:12:57.786135: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-02 15:12:57.810276: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

Visible devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [2]:
# Change the working directoryQ to "/path/to/your/directory"
os.chdir('/home/netdb/project/Name_Disambiguation_BERT')
print("Current directory：", os.getcwd())

Current directory： /home/netdb/project/Name_Disambiguation_BERT


# 1. Preprocessing stage

In [3]:
class settings:
    # Define settings using class attributes
    PROJ_DIR = os.getcwd()  # Get the current working directory as the project directory
    DATA_DIR = join(PROJ_DIR, 'data')
    OUT_DIR = join(PROJ_DIR, 'out')
    EMB_DATA_DIR = join(DATA_DIR, 'emb')
    GLOBAL_DATA_DIR = join(DATA_DIR, 'global')

# Make sure the output directory exists
os.makedirs(settings.OUT_DIR, exist_ok=True)
os.makedirs(settings.EMB_DATA_DIR, exist_ok=True)

In [4]:
class data_utils:
    @staticmethod
    def load_json(rfdir, rfname):
        with codecs.open(join(rfdir, rfname), 'r', encoding='utf-8') as rf:
            return json.load(rf)

    @staticmethod
    def dump_json(obj, wfpath, wfname, indent=None):
        with codecs.open(join(wfpath, wfname), 'w', encoding='utf-8') as wf:
            json.dump(obj, wf, ensure_ascii=False, indent=indent)

    @staticmethod
    def dump_data(obj, wfpath, wfname):
        with open(join(wfpath, wfname), 'wb') as wf:
            pickle.dump(obj, wf)

    @staticmethod
    def load_data(rfpath, rfname):
        with open(join(rfpath, rfname), 'rb') as rf:
            return pickle.load(rf)

    @staticmethod
    def serialize_embedding(embedding):
        return pickle.dumps(embedding)

    @staticmethod
    def deserialize_embedding(s):
        return pickle.loads(s)

# Define Singleton decorator
class Singleton:
    def __init__(self, decorated):
        self._decorated = decorated

    def Instance(self):
        try:
            return self._instance
        except AttributeError:
            self._instance = self._decorated()
            return self._instance

    def __call__(self):
        raise TypeError('Singletons must be accessed through `Instance()`.')

    def __instancecheck__(self, inst):
        return isinstance(inst, self._decorated)

data_utils = data_utils()

In [5]:
map_size = 1099511627776

class LMDBClient(object):
    def __init__(self, name, readonly=False):
        try:
            # Add the database name to the path here
            lmdb_dir = os.path.join(settings.DATA_DIR, 'lmdb', name)
            print(f"LMDB directory: {lmdb_dir}")  # Confirm that the full path is correct
            os.makedirs(lmdb_dir, exist_ok=True)
            self.db = lmdb.open(lmdb_dir, map_size=map_size, readonly=readonly)
        except Exception as e:
            print(f"LMDB initialization error: {e}")
            raise e
            
    def get(self, key):
        try:
            with self.db.begin() as txn:
                value = txn.get(key.encode())
            if value:
                return data_utils.deserialize_embedding(value)
            else:
                return None
        except Exception as e:  
            print(f"Error retrieving key {key}: {e}")
            return None
            
    def get_batch(self, keys):
        values = []
        with self.db.begin() as txn:
            for key in keys:
                value = txn.get(key.encode())
                if value:
                    values.append(data_utils.deserialize_embedding(value))
        return values

    def set(self, key, vector):
        with self.db.begin(write=True) as txn:
            txn.put(key.encode("utf-8"), data_utils.serialize_embedding(vector))

    def set_batch(self, generator):
        with self.db.begin(write=True) as txn:
            for key, vector in generator:
                txn.put(key.encode("utf-8"), data_utils.serialize_embedding(vector))
                print(key, self.get(key))

In [6]:
class string_utils:
    punct = set(u''':!),.:;?.]}¢'"、。〉》」』〕〗〞︰︱︳﹐､﹒
    ﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､￠
    々‖•·ˇˉ―′’”([{£¥'"‵〈《「『〔〖（［｛￡￥〝︵︷︹︻
    ︽︿﹁﹃﹙﹛﹝（｛“‘_…/''')
    stemmer = nltk.stem.PorterStemmer()

    @staticmethod
    def stem(word):
        return string_utils.stemmer.stem(word)

    @staticmethod
    def clean_sentence(text, stemming=False):
        for token in string_utils.punct:
            text = text.replace(token, "")
        words = text.split()
        if stemming:
            stemmed_words = [string_utils.stem(w) for w in words]
            words = stemmed_words
        return " ".join(words)

    @staticmethod
    def clean_name(name):
        if name is None:
            return ""
        x = [k.strip() for k in name.lower().strip().replace(".", " ").replace("-", " ").split()]
        return "_".join(x)

In [7]:
class feature_utils:
    @staticmethod
    def transform_feature(data, f_name, k=1):
        if isinstance(data, str):
            data = data.split()
        assert isinstance(data, list), "Data must be a list"
        features = []
        for d in data:
            features.append("__%s__%s" % (f_name.upper(), d))
        return features

    def extract_common_features(self, item):
        title_features = self.transform_feature(string_utils.clean_sentence(item["title"], stemming=True).lower(), "title")
        keywords_features = []
        keywords = item.get("keywords")
        if keywords:
            keywords_features = self.transform_feature([string_utils.clean_name(k) for k in keywords], 'keyword')
        venue_features = []
        venue_name = item.get('venue', '')
        if len(venue_name) > 2:
            venue_features = self.transform_feature(string_utils.clean_sentence(venue_name.lower()), "venue")
        return title_features, keywords_features, venue_features

    def extract_author_features(self, item, order=None):
        title_features, keywords_features, venue_features = self.extract_common_features(item)
        author_features = []
        for i, author in enumerate(item["authors"]):
            if order is not None and i != order:
                continue
            name_feature = []
            org_features = []
            org_name = string_utils.clean_name(author.get("org", ""))
            if len(org_name) > 2:
                org_features.extend(self.transform_feature(org_name, "org"))
            for j, coauthor in enumerate(item["authors"]):
                if i == j:
                    continue
                coauthor_name = coauthor.get("name", "")
                coauthor_org = string_utils.clean_name(coauthor.get("org", ""))
                if len(coauthor_name) > 2:
                    name_feature.extend(
                        self.transform_feature([string_utils.clean_name(coauthor_name)], "name")
                    )
                if len(coauthor_org) > 2:
                    org_features.extend(
                        self.transform_feature(string_utils.clean_sentence(coauthor_org.lower()), "org")
                    )
            author_features.append(
                name_feature + org_features + title_features + keywords_features + venue_features
            )
        author_features = list(chain.from_iterable(author_features))
        return author_features

feature_utils = feature_utils()

In [8]:
# Configuration log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
            
# Dimensions of embedding vectors, BERT's specific model has fixed dimensions, the 'bert-base-uncased' used is 768
EMB_DIM = 768

@Singleton
class EmbeddingModel:
    def __init__(self, name="bert-base-uncased"):
        self.model_name = name
        self.tokenizer = BertTokenizer.from_pretrained(name)
        self.model = BertModel.from_pretrained(name)
        self.model.eval()  # Set the model to evaluation mode

    @lru_cache()
    def get_embedding(self, text):
        inputs = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        # Take the first token (CLS token) of the last layer’s hidden state as the representation of the sentence
        return outputs.last_hidden_state[:, 0, :].squeeze()

    def process_data_from_lmdb(self):
        LMDB_NAME = 'pub_authors_feature'
        lc = LMDBClient(LMDB_NAME)
        author_cnt = 0
        data = []
        with lc.db.begin() as txn:
            for k in txn.cursor():
                author_feature = data_utils.deserialize_embedding(k[1])
                if author_cnt % 10000 == 0:
                    print(author_cnt, author_feature[0])
                author_cnt += 1
                random.shuffle(author_feature)
                # BERT embeddings can be obtained using each author feature here
                embedding = self.get_embedding(" ".join(author_feature))
                data.append(embedding)
        return data

    def save(self, wf_name):
        # Save model and tokenizer
        model_path = join(settings.EMB_DATA_DIR, '{}_model'.format(wf_name))
        tokenizer_path = join(settings.EMB_DATA_DIR, '{}_tokenizer'.format(wf_name))
        self.model.save_pretrained(model_path)
        self.tokenizer.save_pretrained(tokenizer_path)

    def load(self, name):
        # Load model and tokenizer
        model_path = join(settings.EMB_DATA_DIR, '{}_model'.format(name))
        tokenizer_path = join(settings.EMB_DATA_DIR, '{}_tokenizer'.format(name))
        self.model = BertModel.from_pretrained(model_path)
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
        self.model.eval()

    def project_embedding(self, tokens, idf=None):
        """
        Weighted average token embedding
        :param tokens: input word list
        :param idf: IDF dictionary
        :return: Obtained weighted average embedding
        """
        vectors = []
        sum_weight = 0
        for token in tokens:
            embedding = self.get_embedding(token)
            weight = idf[token] if idf and token in idf else 1
            v = embedding * weight
            vectors.append(v)
            sum_weight += weight
        if len(vectors) == 0:
            print('all tokens not in BERT models')
            return None
        emb = torch.sum(torch.stack(vectors), dim=0)
        emb /= sum_weight
        return emb.cpu().numpy()

In [9]:
start_time = datetime.now()

def dump_author_features_to_file():
    """
    generate author features by raw publication data and dump to files
    author features are defined by his/her paper attributes excluding the author's name
    """
    pubs_dict = data_utils.load_json(settings.GLOBAL_DATA_DIR, 'pubs_raw.json')
    print('n_papers', len(pubs_dict))
    wf = codecs.open(join(settings.GLOBAL_DATA_DIR, 'author_features.txt'), 'w', encoding='utf-8')
    for i, pid in enumerate(pubs_dict):
        if i % 1000 == 0:
            print(i, datetime.now()-start_time)
        paper = pubs_dict[pid]
        if "title" not in paper or "authors" not in paper:
            continue
        if len(paper["authors"]) > 30:
            print(i, pid, len(paper["authors"]))
        if len(paper["authors"]) > 100:
            continue
        n_authors = len(paper.get('authors', []))
        for j in range(n_authors):
            author_feature = feature_utils.extract_author_features(paper, j)
            aid = '{}-{}'.format(pid, j)
            wf.write(aid + '\t' + ' '.join(author_feature) + '\n')
    wf.close()

def dump_author_features_to_cache():
    """
    dump author features to lmdb
    """
    LMDB_NAME = 'pub_authors_feature'
    lc = LMDBClient(LMDB_NAME)
    with codecs.open(join(settings.GLOBAL_DATA_DIR, 'author_features.txt'), 'r', encoding='utf-8') as rf:
        for i, line in enumerate(rf):
            if i % 1000 == 0:
                print('line', i)
            items = line.rstrip().split('\t')
            pid_order = items[0]
            author_features = items[1].split()
            lc.set(pid_order, author_features)

def cal_feature_idf():
    """
    calculate word IDF (Inverse document frequency) using publication data
    """
    feature_dir = join(settings.DATA_DIR, 'global')
    counter = dd(int)
    cnt = 0
    LMDB_NAME = 'pub_authors_feature'
    lc = LMDBClient(LMDB_NAME)
    author_cnt = 0
    with lc.db.begin() as txn:
        for k in txn.cursor():
            features = data_utils.deserialize_embedding(k[1])
            if author_cnt % 10000 == 0:
                print(author_cnt, features[0], counter.get(features[0]))
            author_cnt += 1
            for f in features:
                cnt += 1
                counter[f] += 1
    idf = {}
    for k in counter:
        idf[k] = math.log(cnt / counter[k])
    data_utils.dump_data(dict(idf), feature_dir, "feature_idf.pkl")

def dump_author_embs():
    """
    dump author embedding to lmdb
    author embedding is calculated by weighted-average of word vectors with IDF
    """
    emb_model = EmbeddingModel.Instance()
    idf = data_utils.load_data(settings.GLOBAL_DATA_DIR, 'feature_idf.pkl')
    print('idf loaded')
    LMDB_NAME_FEATURE = 'pub_authors_feature'
    lc_feature = LMDBClient(LMDB_NAME_FEATURE)
    LMDB_NAME_EMB = "author_100_emb_weighted"
    lc_emb = LMDBClient(LMDB_NAME_EMB)
    cnt = 0
    with lc_feature.db.begin() as txn:
        for k in txn.cursor():
            if cnt % 1000 == 0:
                print('cnt', cnt, datetime.now()-start_time)
            cnt += 1
            pid_order = k[0].decode('utf-8')
            features = data_utils.deserialize_embedding(k[1])
            cur_emb = emb_model.project_embedding(features, idf)
            if cur_emb is not None:
                lc_emb.set(pid_order, cur_emb)

In [10]:
dump_author_features_to_file()  # Generate author characteristics and save to file
dump_author_features_to_cache()  # Export author characteristics to LMDB cache

n_papers 203078
0 0:00:47.192593
252 5b5433e5e1cd8e4e15f76e38 42
358 5b5433e8e1cd8e4e15fd6be3 49
394 5b5433e9e1cd8e4e15ff4e38 56
748 5b5433e6e1cd8e4e15fa9f20 41
750 5b5433f7e1cd8e4e151f569d 44
978 5b5433f0e1cd8e4e15104ca3 43
1000 0:00:47.611963
1351 5b5433ebe1cd8e4e150513f2 47
2000 0:00:47.887945
3000 0:00:48.141939
4000 0:00:48.374982
4720 5b5433e3e1cd8e4e15f469df 46
5000 0:00:48.685206
5721 5b5433ede1cd8e4e1509e1c0 36
5994 5b5433e6e1cd8e4e15fadcf2 32
5997 5b5433f5e1cd8e4e151b4445 33
6000 0:00:49.009975
7000 0:00:49.290679
7034 5b5433e8e1cd8e4e15fd693f 31
7881 5b5433eae1cd8e4e1502ca53 34
7882 5b5433ede1cd8e4e1508f522 33
8000 0:00:49.626557
9000 0:00:49.926262
9867 5b5433e9e1cd8e4e1500318e 32
10000 0:00:50.247870
10051 5b5433eae1cd8e4e1501f81a 50
10062 5b5433ebe1cd8e4e15053cfd 43
10936 5b5433f1e1cd8e4e151129cc 92
10966 5b5433eae1cd8e4e1502a624 84
11000 0:00:50.760103
11580 5b5433e4e1cd8e4e15f6718e 100
11670 5b5433e9e1cd8e4e15007eb9 59
12000 0:00:51.072238
12375 5b5433e4e1cd8e4e15f4ba95

In [11]:
# Calculate IDF and export
cal_feature_idf()

LMDB directory: /home/netdb/project/Name_Disambiguation_BERT/data/lmdb/pub_authors_feature
0 __NAME__guoliang_li None
10000 __NAME__xiu_ping_zhang 4
20000 __NAME__ning_hu 1
30000 __NAME__lin_huang 78
40000 __NAME__ming_fu_zhao None
50000 __NAME__gentao_cao 6
60000 __NAME__m_wu 104
70000 __NAME__fenghuang_zhan 5
80000 __NAME__dong_wang 125
90000 __NAME__guangpin_chen None
100000 __NAME__xiangrui_jiang 12
110000 __NAME__xiao_huang 1
120000 __NAME__liqiang_cui None
130000 __NAME__jianjun_yang 240
140000 __NAME__zhuyuan_wang 60
150000 __NAME__baocheng_hu 8
160000 __NAME__lin_wang 283
170000 __NAME__kui_jiao 10
180000 __NAME__rong_yu 275
190000 __NAME__yehui_yang 2
200000 __NAME__yu_guo 730
210000 __NAME__li_dai 17
220000 __NAME__jianhua_zhang 50
230000 __NAME__xian_du 2
240000 __NAME__yi_feng 70
250000 __NAME__xian_jin_tang 2
260000 __NAME__rui_zhou 83
270000 __NAME__xiqing_chen 2
280000 __NAME__w_chen 196
290000 __NAME__linghong_liang 12
300000 __NAME__zhen_yu_li 383
310000 __NAME__yulei_

In [12]:
# Export author embed
dump_author_embs()

idf loaded
LMDB directory: /home/netdb/project/Name_Disambiguation_BERT/data/lmdb/pub_authors_feature
LMDB directory: /home/netdb/project/Name_Disambiguation_BERT/data/lmdb/author_100_emb_weighted
cnt 0 3:27:15.822908
cnt 1000 3:29:32.345855
cnt 2000 3:31:46.393450
cnt 3000 3:33:59.199200
cnt 4000 3:36:17.853232
cnt 5000 3:38:27.516465
cnt 6000 3:40:39.877542
cnt 7000 3:42:52.218178
cnt 8000 3:44:53.406388
cnt 9000 3:46:44.224302
cnt 10000 3:55:20.407536
cnt 11000 3:57:16.712885
cnt 12000 3:59:18.276801
cnt 13000 4:01:19.076367
cnt 14000 4:03:15.867377
cnt 15000 4:05:15.824586
cnt 16000 4:07:13.426399
cnt 17000 4:09:14.988720
cnt 18000 4:11:23.659170
cnt 19000 4:13:27.046744
cnt 20000 4:15:41.153685
cnt 21000 4:25:55.195940
cnt 22000 4:28:24.135301
cnt 23000 4:30:55.680782
cnt 24000 4:33:26.945444
cnt 25000 4:35:58.939260
cnt 26000 4:38:24.044127
cnt 27000 4:40:55.806190
cnt 28000 4:43:28.221740
cnt 29000 4:45:52.438826
cnt 30000 4:48:23.388849
cnt 31000 4:50:54.277282
cnt 32000 4:53:2

In [13]:
# Print completion information
print('done', datetime.now() - start_time)

done 1 day, 23:11:40.401152


# 2. Global model_Gen_Training stage

In [9]:
class TripletsGenerator:
    name2pubs_train = {}
    name2pubs_test = {}
    names_train = None
    names_test = None
    n_pubs_train = None
    n_pubs_test = None
    pids_train = []
    pids_test = []
    n_triplets = 0
    batch_size = 100000

    def __init__(self, train_scale=10000):
        self.prepare_data()
        self.save_size = train_scale
        self.idf = data_utils.load_data(settings.GLOBAL_DATA_DIR, 'feature_idf.pkl')

    def prepare_data(self):
        self.name2pubs_train = data_utils.load_json(settings.GLOBAL_DATA_DIR, 'name_to_pubs_train_500.json')  # for test
        self.name2pubs_test = data_utils.load_json(settings.GLOBAL_DATA_DIR, 'name_to_pubs_test_100.json')
        self.names_train = self.name2pubs_train.keys()
        print('names train', len(self.names_train))
        self.names_test = self.name2pubs_test.keys()
        print('names test', len(self.names_test))
        assert not set(self.names_train).intersection(set(self.names_test))
        for name in self.names_train:
            name_pubs_dict = self.name2pubs_train[name]
            for aid in name_pubs_dict:
                self.pids_train += name_pubs_dict[aid]
        random.shuffle(self.pids_train)
        self.n_pubs_train = len(self.pids_train)
        print('pubs2train', self.n_pubs_train)

        for name in self.names_test:
            name_pubs_dict = self.name2pubs_test[name]
            for aid in name_pubs_dict:
                self.pids_test += name_pubs_dict[aid]
        random.shuffle(self.pids_test)
        self.n_pubs_test = len(self.pids_test)
        print('pubs2test', self.n_pubs_test)

    def gen_neg_pid(self, not_in_pids, role='train'):
        if role == 'train':
            sample_from_pids = self.pids_train
        else:
            sample_from_pids = self.pids_test
        while True:
            idx = random.randint(0, len(sample_from_pids)-1)
            pid = sample_from_pids[idx]
            if pid not in not_in_pids:
                return pid

    def sample_triplet_ids(self, task_q, role='train', N_PROC=8):
        n_sample_triplets = 0
        if role == 'train':
            names = self.names_train
            name2pubs = self.name2pubs_train
        else:  # test
            names = self.names_test
            name2pubs = self.name2pubs_test
            self.save_size = 200000  # test save size
        for name in names:
            name_pubs_dict = name2pubs[name]
            for aid in name_pubs_dict:
                pub_items = name_pubs_dict[aid]
                if len(pub_items) == 1:
                    continue
                pids = pub_items
                cur_n_pubs = len(pids)
                random.shuffle(pids)
                for i in range(cur_n_pubs):
                    pid1 = pids[i]  # pid

                    # batch samples
                    n_samples_anchor = min(6, cur_n_pubs)
                    idx_pos = random.sample(range(cur_n_pubs), n_samples_anchor)
                    for ii, i_pos in enumerate(idx_pos):
                        if i_pos != i:
                            if n_sample_triplets % 100 == 0:
                                # print('sampled triplet ids', n_sample_triplets)
                                pass
                            pid_pos = pids[i_pos]
                            pid_neg = self.gen_neg_pid(pids, role)
                            n_sample_triplets += 1
                            task_q.put((pid1, pid_pos, pid_neg))

                            if n_sample_triplets >= self.save_size:
                                for j in range(N_PROC):
                                    task_q.put((None, None, None))
                                return
        for j in range(N_PROC):
            task_q.put((None, None, None))

    def gen_emb_mp(self, task_q, emb_q):
        while True:
            pid1, pid_pos, pid_neg = task_q.get()
            if pid1 is None:
                break
            emb1 = lc.get(pid1)
            emb_pos = lc.get(pid_pos)
            emb_neg = lc.get(pid_neg)
            if emb1 is not None and emb_pos is not None and emb_neg is not None:
                emb_q.put((emb1, emb_pos, emb_neg))
        emb_q.put((False, False, False))

    def gen_triplets_mp(self, role='train'):
        N_PROC = 8

        task_q = mp.Queue(N_PROC * 6)
        emb_q = mp.Queue(1000)

        producer_p = mp.Process(target=self.sample_triplet_ids, args=(task_q, role, N_PROC))
        consumer_ps = [mp.Process(target=self.gen_emb_mp, args=(task_q, emb_q)) for _ in range(N_PROC)]
        producer_p.start()
        [p.start() for p in consumer_ps]

        cnt = 0

        while True:
            if cnt % 1000 == 0:
                print('get', cnt, datetime.now()-start_time)
            emb1, emb_pos, emb_neg = emb_q.get()
            if emb1 is False:
                producer_p.terminate()
                producer_p.join()
                [p.terminate() for p in consumer_ps]
                [p.join() for p in consumer_ps]
                break
            cnt += 1
            yield (emb1, emb_pos, emb_neg)

    def dump_triplets(self, role='train'):
        triplets = self.gen_triplets_mp(role)
        if role == 'train':
            out_dir = join(settings.OUT_DIR, 'triplets-{}'.format(self.save_size))
        else:
            out_dir = join(settings.OUT_DIR, 'test-triplets')
        os.makedirs(out_dir, exist_ok=True)
        anchor_embs = []
        pos_embs = []
        neg_embs = []
        f_idx = 0
        for i, t in enumerate(triplets):
            if i % 100 == 0:
                print(i, datetime.now()-start_time)
            emb_anc, emb_pos, emb_neg = t[0], t[1], t[2]
            anchor_embs.append(emb_anc)
            pos_embs.append(emb_pos)
            neg_embs.append(emb_neg)
            if len(anchor_embs) == self.batch_size:
                data_utils.dump_data(anchor_embs, out_dir, 'anchor_embs_{}_{}.pkl'.format(role, f_idx))
                data_utils.dump_data(pos_embs, out_dir, 'pos_embs_{}_{}.pkl'.format(role, f_idx))
                data_utils.dump_data(neg_embs, out_dir, 'neg_embs_{}_{}.pkl'.format(role, f_idx))
                f_idx += 1
                anchor_embs = []
                pos_embs = []
                neg_embs = []
        if anchor_embs:
            data_utils.dump_data(anchor_embs, out_dir, 'anchor_embs_{}_{}.pkl'.format(role, f_idx))
            data_utils.dump_data(pos_embs, out_dir, 'pos_embs_{}_{}.pkl'.format(role, f_idx))
            data_utils.dump_data(neg_embs, out_dir, 'neg_embs_{}_{}.pkl'.format(role, f_idx))
        print('dumped')

In [15]:
LMDB_NAME = "author_100_emb_weighted"
lc = LMDBClient(LMDB_NAME)
start_time = datetime.now()

"""
This class generates triplets of author embeddings to train global model
"""

data_gen = TripletsGenerator(train_scale=1000000)
data_gen.dump_triplets(role='train')
data_gen.dump_triplets(role='test')

LMDB directory: /home/netdb/project/Name_Disambiguation_BERT/data/lmdb/author_100_emb_weighted
names train 500
names test 100
pubs2train 173698
pubs2test 35129
get 0 0:00:00.556977
0 0:00:00.557749
100 0:00:00.561587
200 0:00:00.563693
300 0:00:00.565761
400 0:00:00.567806
500 0:00:00.569664
600 0:00:00.571510
700 0:00:00.573472
800 0:00:00.575406
900 0:00:00.577342
get 1000 0:00:00.578962
1000 0:00:00.578991
1100 0:00:00.580678
1200 0:00:00.582281
1300 0:00:00.584125
1400 0:00:00.585713
1500 0:00:00.587355
1600 0:00:00.589083
1700 0:00:00.590737
1800 0:00:00.592531
1900 0:00:00.594376
get 2000 0:00:00.595908
2000 0:00:00.595934
2100 0:00:00.597589
2200 0:00:00.599234
2300 0:00:00.600842
2400 0:00:00.602513
2500 0:00:00.604096
2600 0:00:00.605690
2700 0:00:00.607277
2800 0:00:00.608876
2900 0:00:00.610608
get 3000 0:00:00.612125
3000 0:00:00.612148
3100 0:00:00.613761
3200 0:00:00.615347
3300 0:00:00.616928
3400 0:00:00.618698
3500 0:00:00.620229
3600 0:00:00.621989
3700 0:00:00.624000

# 3. Global model_Global model stage

In [10]:
class Triplet:
    @staticmethod
    def l2Norm(x):
        return tf.math.l2_normalize(x, axis=-1)
    
    @staticmethod
    def euclidean_distance(vects):
        x, y = vects
        return tf.sqrt(tf.maximum(tf.reduce_sum(tf.square(x - y), axis=1, keepdims=True), K.epsilon()))
    
    @staticmethod
    def triplet_loss(_, y_pred):
        # Make sure that the shape of y_pred is correctly understood as [batch_size, 2], the first one is the positive sample distance and the second one is the negative sample distance
        margin = 1.0
        pos_dist = y_pred[:, 0]  # Positive sample distance
        neg_dist = y_pred[:, 1]  # Negative sample distance
        # Calculate triplet loss
        loss = K.mean(K.maximum(pos_dist - neg_dist + margin, 0.0))
        return loss
    
    @staticmethod
    def accuracy(_, y_pred):
        # Judging accuracy based on distance
        pos_dist = y_pred[:, 0]
        neg_dist = y_pred[:, 1]
        return K.mean(pos_dist < neg_dist)

triplet = Triplet()

In [11]:
class eval_utils:
    @staticmethod
    def pairwise_precision_recall_f1(preds, truths):
        tp = 0
        fp = 0
        fn = 0
        n_samples = len(preds)
        for i in range(n_samples - 1):
            pred_i = preds[i]
            for j in range(i + 1, n_samples):
                pred_j = preds[j]
                if pred_i == pred_j:
                    if truths[i] == truths[j]:
                        tp += 1
                    else:
                        fp += 1
                elif truths[i] == truths[j]:
                    fn += 1
        tp_plus_fp = tp + fp
        tp_plus_fn = tp + fn
        if tp_plus_fp == 0:
            precision = 0.
        else:
            precision = tp / tp_plus_fp
        if tp_plus_fn == 0:
            recall = 0.
        else:
            recall = tp / tp_plus_fn

        if not precision or not recall:
            f1 = 0.
        else:
            f1 = (2 * precision * recall) / (precision + recall)
        return precision, recall, f1

    @staticmethod
    def cal_f1(prec, rec):
        return 2 * prec * rec / (prec + rec) if (prec + rec) else 0

    @staticmethod
    def get_hidden_output(model, inp):
        # The first input of the original model is the anchor input
        anchor_input = model.input[0]  # Get the anchor point input of the original model
        intermediate_layer_model = Model(inputs=anchor_input, outputs=model.layers[5].output)
        # Make sure the input provided is a list, even if it only has one element
        intermediate_output = intermediate_layer_model([inp])
        return intermediate_output
        
    @staticmethod
    def predict(anchor_emb, test_embs):
        score1 = np.linalg.norm(anchor_emb - test_embs[0])
        score2 = np.linalg.norm(anchor_emb - test_embs[1])
        return [score1, score2]

    @staticmethod
    def full_auc(model, test_triplets):
    
        """
        Measure AUC for model and ground truth on all items.
    
        Returns:
        - float AUC
        """
    
        grnds = []
        preds = []
        preds_before = []
        embs_anchor, embs_pos, embs_neg = test_triplets
    
        inter_embs_anchor = eval_utils.get_hidden_output(model, embs_anchor)
        inter_embs_pos = eval_utils.get_hidden_output(model, embs_pos)
        inter_embs_neg = eval_utils.get_hidden_output(model, embs_neg)
        # print(inter_embs_pos.shape)
    
        accs = []
        accs_before = []
    
        for i, e in enumerate(inter_embs_anchor):
            if i % 10000 == 0:
                print('test', i)
    
            emb_anchor = e
            emb_pos = inter_embs_pos[i]
            emb_neg = inter_embs_neg[i]
            test_embs = np.array([emb_pos, emb_neg])
    
            emb_anchor_before = embs_anchor[i]
            emb_pos_before = embs_pos[i]
            emb_neg_before = embs_neg[i]
            test_embs_before = np.array([emb_pos_before, emb_neg_before])
    
            predictions = eval_utils.predict(emb_anchor, test_embs)
            predictions_before = eval_utils.predict(emb_anchor_before, test_embs_before)

            acc_before = 1 if predictions_before[0] < predictions_before[1] else 0
            acc = 1 if predictions[0] < predictions[1] else 0
            accs_before.append(acc_before)
            accs.append(acc)
    
            grnd = [0, 1]
            grnds += grnd
            preds += predictions
            preds_before += predictions_before
    
        auc_before = roc_auc_score(grnds, preds_before)
        auc = roc_auc_score(grnds, preds)
        print('test accuracy before', np.mean(accs_before))
        print('test accuracy after', np.mean(accs))
    
        print('test AUC before', auc_before)
        print('test AUC after', auc)
        return auc
        
eval_utils = eval_utils()

In [12]:
# Define a custom layer to replace the Lambda layer
@register_keras_serializable()
class custom_stack_layer(Layer):
    def __init__(self, **kwargs):
        super(custom_stack_layer, self).__init__(**kwargs)

    def call(self, inputs):
        pos_dist, neg_dist = inputs
        return tf.stack([pos_dist, neg_dist], axis=1)

    def compute_output_shape(self, input_shape):
        shape = list(input_shape[0])
        return (shape[0], 2, 1)

    def get_config(self):
        config = super(custom_stack_layer, self).get_config()
        return config
        
@register_keras_serializable()
class custom_lambda_layer(Layer):
    def __init__(self, **kwargs):
        super(custom_lambda_layer, self).__init__(**kwargs)

    def call(self, inputs):
        pos_dist, neg_dist = inputs
        return tf.sqrt(tf.maximum(tf.reduce_sum(tf.square(inputs[0] - inputs[1]), axis=1, keepdims=True), tf.keras.backend.epsilon()))

    def get_config(self):
        config = super(custom_lambda_layer, self).get_config()
        return config
        
@register_keras_serializable()
class l2_Norm_layer(Layer):
    def __init__(self, **kwargs):
        super(l2_Norm_layer, self).__init__(**kwargs)

    def call(self, inputs):
        return triplet.l2Norm(inputs)

In [13]:
"""
global metric learning model
"""

EMB_DIM = 768
#@tf.keras.utils.register_keras_serializable()
#@staticmethod
def cal_output_shape(input_shape):
    shape = list(input_shape[0])
    assert len(shape) == 2  # only valid for 2D tensors
    shape[-1] *= 2
    return tuple(shape)
    
class GlobalTripletModel:

    def __init__(self, data_scale, EMB_DIM=768):
        self.data_scale = data_scale
        self.EMB_DIM = EMB_DIM
        self.train_triplets_dir = join(settings.OUT_DIR, 'triplets-{}'.format(self.data_scale))
        self.test_triplets_dir = join(settings.OUT_DIR, 'test-triplets')
        self.train_triplet_files_num = self.get_triplets_files_num(self.train_triplets_dir)
        self.test_triplet_files_num = self.get_triplets_files_num(self.test_triplets_dir)
        print('test file num', self.test_triplet_files_num)

    @staticmethod
    def get_triplets_files_num(path_dir):
        files = []
        for f in os.listdir(path_dir):
            if f.startswith('anchor_embs_'):
                files.append(f)
        return len(files)

    def load_batch_triplets(self, f_idx, role='train'):
        if role == 'train':
            cur_dir = self.train_triplets_dir
        else:
            cur_dir = self.test_triplets_dir
        X1 = data_utils.load_data(cur_dir, 'anchor_embs_{}_{}.pkl'.format(role, f_idx))
        X2 = data_utils.load_data(cur_dir, 'pos_embs_{}_{}.pkl'.format(role, f_idx))
        X3 = data_utils.load_data(cur_dir, 'neg_embs_{}_{}.pkl'.format(role, f_idx))
        return X1, X2, X3

    def load_triplets_data(self, role='train'):
        X1 = np.empty([0, EMB_DIM])
        X2 = np.empty([0, EMB_DIM])
        X3 = np.empty([0, EMB_DIM])
        if role == 'train':
            f_num = self.train_triplet_files_num
        else:
            f_num = self.test_triplet_files_num
        for i in range(f_num):
            print('load', i)
            x1_batch, x2_batch, x3_batch = self.load_batch_triplets(i, role)
            p = np.random.permutation(len(x1_batch))
            x1_batch = np.array(x1_batch)[p]
            x2_batch = np.array(x2_batch)[p]
            x3_batch = np.array(x3_batch)[p]
            X1 = np.concatenate((X1, x1_batch))
            X2 = np.concatenate((X2, x2_batch))
            X3 = np.concatenate((X3, x3_batch))
        return X1, X2, X3

    @staticmethod
    def create_triplet_model(EMB_DIM=768):
        emb_anchor = Input(shape=(EMB_DIM,), name='anchor_input')
        emb_pos = Input(shape=(EMB_DIM,), name='pos_input')
        emb_neg = Input(shape=(EMB_DIM,), name='neg_input')

        # shared layers
        layer1 = Dense(128, activation='relu', name='first_emb_layer')
        layer2 = Dense(64, activation='relu', name='last_emb_layer')
        norm_layer = l2_Norm_layer(name='norm_layer')
        
        encoded_emb = norm_layer(layer2(layer1(emb_anchor)))
        encoded_emb_pos = norm_layer(layer2(layer1(emb_pos)))
        encoded_emb_neg = norm_layer(layer2(layer1(emb_neg)))

        lambda_layer = custom_lambda_layer(name='distance_layer')
        pos_dist = lambda_layer([encoded_emb, encoded_emb_pos])
        neg_dist = lambda_layer([encoded_emb, encoded_emb_neg])

        stack_layer = custom_stack_layer(name='stack_layer')
        stacked_dists = stack_layer([pos_dist, neg_dist])
        
        model = Model(inputs=[emb_anchor, emb_pos, emb_neg], outputs=stacked_dists)
        model.compile(loss=triplet.triplet_loss, optimizer=Adam(learning_rate=0.01), metrics=[triplet.accuracy])

       
    
        inter_layer = Model(inputs=model.inputs, outputs=model.get_layer('norm_layer').output)
    
        return model, inter_layer

    def load_triplets_model(self):
        model_dir = join(settings.OUT_DIR, 'model')
        model_filename = os.path.join(model_dir, 'model-triplets-{}.json'.format(self.data_scale))

        with open(model_filename, 'r') as json_file:
            model_json_string = json_file.read() 

    
        # Define custom_objects dictionary, containing all custom functions
        custom_objects = {
            'custom_lambda_layer': custom_lambda_layer,
            'custom_stack_layer': custom_stack_layer,
            'l2Norm': triplet.l2Norm,
            'euclidean_distance': triplet.euclidean_distance,
            'triplet_loss': triplet.triplet_loss,
            'accuracy': triplet.accuracy,
            'cal_output_shape': cal_output_shape
        }

        # Load the model using the read JSON string and custom objects
        loaded_model = model_from_json(model_json_string, custom_objects=custom_objects)
        weights_filename = os.path.join(model_dir, 'model-triplets-{}.weights.h5'.format(self.data_scale))
        loaded_model.load_weights(weights_filename)
        return loaded_model

    def train_triplets_model(self):
        X1, X2, X3 = self.load_triplets_data()
        n_triplets = len(X1)
        print('loaded')
        model, inter_model = self.create_triplet_model()
        # print(model.summary())

        X_anchor, X_pos, X_neg = X1, X2, X3
        X = {'anchor_input': X_anchor, 'pos_input': X_pos, 'neg_input': X_neg}
        model.fit(X, np.ones((n_triplets, 2)), batch_size=64, epochs=5, shuffle=True, validation_split=0.2)

        model_json = model.to_json()
        model_dir = join(settings.OUT_DIR, 'model')
        os.makedirs(model_dir, exist_ok=True)
        with open(join(model_dir, 'model-triplets-{}.json'.format(self.data_scale)), 'w') as wf:
            wf.write(model_json)
        model.save_weights(join(model_dir, 'model-triplets-{}.weights.h5'.format(self.data_scale)))

        test_triplets = self.load_triplets_data(role='test')
        auc_score = eval_utils.full_auc(model, test_triplets)
        # print('AUC', auc_score)

        loaded_model = self.load_triplets_model()
        print('triplets model loaded')
        auc_score = eval_utils.full_auc(loaded_model, test_triplets)

    def evaluate_triplet_model(self):
        test_triplets = self.load_triplets_data(role='test')
        loaded_model = self.load_triplets_model()
        print('triplets model loaded')
        auc_score = eval_utils.full_auc(loaded_model, test_triplets)

In [14]:
global_model = GlobalTripletModel(data_scale=1000000)
global_model.train_triplets_model()
print('done')

test file num 2
load 0
load 1
load 2
load 3
load 4
load 5
load 6
load 7
loaded
Epoch 1/5
 111/9929 [..............................] - ETA: 13s - loss: 0.7284 - accuracy: 0.7248  

2024-05-02 15:13:47.453468: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f2a50008f60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-05-02 15:13:47.453498: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Host, Default Version
2024-05-02 15:13:47.457975: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-02 15:13:47.494699: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


9929/9929 [==============================] - 18s 2ms/step - loss: 0.5473 - accuracy: 0.7858 - val_loss: 0.5346 - val_accuracy: 0.7844
Epoch 2/5
9929/9929 [==============================] - 15s 2ms/step - loss: 0.5440 - accuracy: 0.7831 - val_loss: 0.8255 - val_accuracy: 0.6040
Epoch 3/5
9929/9929 [==============================] - 16s 2ms/step - loss: 0.5660 - accuracy: 0.7664 - val_loss: 0.6249 - val_accuracy: 0.7283
Epoch 4/5
9929/9929 [==============================] - 15s 2ms/step - loss: 0.6273 - accuracy: 0.7315 - val_loss: 0.6158 - val_accuracy: 0.7345
Epoch 5/5
9929/9929 [==============================] - 16s 2ms/step - loss: 0.6189 - accuracy: 0.7338 - val_loss: 0.6291 - val_accuracy: 0.7116
load 0
load 1
test 0
test 10000
test 20000
test 30000
test 40000
test 50000
test 60000
test 70000
test 80000
test 90000
test 100000
test 110000
test 120000
test 130000
test 140000
test 150000
test 160000
test accuracy before 0.7387686779935374
test accuracy after 0.7289889817466109
test AU

# 4. Global model_Prepare_local_data stage

In [15]:
# IDF_THRESHOLD = 32  # small data
IDF_THRESHOLD = 10


def dump_inter_emb():
    """
    dump hidden embedding via trained global model for local model to use
    """
    LMDB_NAME = "author_100_emb_weighted"
    lc_input = LMDBClient(LMDB_NAME)
    INTER_LMDB_NAME = 'author_triplets.emb'
    lc_inter = LMDBClient(INTER_LMDB_NAME)
    global_model = GlobalTripletModel(data_scale=1000000)
    trained_global_model = global_model.load_triplets_model()
    name_to_pubs_test = data_utils.load_json(settings.GLOBAL_DATA_DIR, 'name_to_pubs_test_100.json')
    for name in name_to_pubs_test:
        print('name', name)
        name_data = name_to_pubs_test[name]
        embs_input = []
        pids = []
        for i, aid in enumerate(name_data.keys()):
            if len(name_data[aid]) < 5:  # n_pubs of current author is too small
                continue
            for pid in name_data[aid]:
                cur_emb = lc_input.get(pid)
                if cur_emb is None:
                    continue
                embs_input.append(cur_emb)
                pids.append(pid)
        embs_input = np.stack(embs_input)
        inter_embs = eval_utils.get_hidden_output(trained_global_model, embs_input)
        for i, pid_ in enumerate(pids):
            lc_inter.set(pid_, inter_embs[i])


def gen_local_data(idf_threshold=10):
    """
    generate local data (including paper features and paper network) for each associated name
    :param idf_threshold: threshold for determining whether there exists an edge between two papers (for this demo we set 29)
    """
    name_to_pubs_test = data_utils.load_json(settings.GLOBAL_DATA_DIR, 'name_to_pubs_test_100.json')
    idf = data_utils.load_data(settings.GLOBAL_DATA_DIR, 'feature_idf.pkl')
    INTER_LMDB_NAME = 'author_triplets.emb'
    lc_inter = LMDBClient(INTER_LMDB_NAME)
    LMDB_AUTHOR_FEATURE = "pub_authors_feature"
    lc_feature = LMDBClient(LMDB_AUTHOR_FEATURE)
    graph_dir = join(settings.DATA_DIR, 'local', 'graph-{}'.format(idf_threshold))
    os.makedirs(graph_dir, exist_ok=True)
    for i, name in enumerate(name_to_pubs_test):
        print(i, name)
        cur_person_dict = name_to_pubs_test[name]
        pids_set = set()
        pids = []
        pids2label = {}

        # generate content
        wf_content = open(join(graph_dir, '{}_pubs_content.txt'.format(name)), 'w')
        for i, aid in enumerate(cur_person_dict):
            items = cur_person_dict[aid]
            if len(items) < 5:
                continue
            for pid in items:
                pids2label[pid] = aid
                pids.append(pid)
        shuffle(pids)
        for pid in pids:
            cur_pub_emb = lc_inter.get(pid)
            if cur_pub_emb is not None:
                cur_pub_emb = list(map(str, cur_pub_emb))
                pids_set.add(pid)
                wf_content.write('{}\t'.format(pid))
                wf_content.write('\t'.join(cur_pub_emb))
                wf_content.write('\t{}\n'.format(pids2label[pid]))
        wf_content.close()

        # generate network
        pids_filter = list(pids_set)
        n_pubs = len(pids_filter)
        print('n_pubs', n_pubs)
        wf_network = open(join(graph_dir, '{}_pubs_network.txt'.format(name)), 'w')
        for i in range(n_pubs-1):
            if i % 10 == 0:
                print(i)
            author_feature1 = set(lc_feature.get(pids_filter[i]))
            for j in range(i+1, n_pubs):
                author_feature2 = set(lc_feature.get(pids_filter[j]))
                common_features = author_feature1.intersection(author_feature2)
                idf_sum = 0
                for f in common_features:
                    idf_sum += idf.get(f, idf_threshold)
                    # print(f, idf.get(f, idf_threshold))
                if idf_sum >= idf_threshold:
                    wf_network.write('{}\t{}\n'.format(pids_filter[i], pids_filter[j]))
        wf_network.close()

In [16]:
dump_inter_emb()
gen_local_data(idf_threshold=IDF_THRESHOLD)
print('done')

LMDB directory: /home/netdb/project/Name_Disambiguation_BERT/data/lmdb/author_100_emb_weighted
LMDB directory: /home/netdb/project/Name_Disambiguation_BERT/data/lmdb/author_triplets.emb
test file num 2
name hongbin_liang
name j_yu
name s_yu
name he_ping_li
name song_chen
name jia_xu
name lixin_tang
name guotong_du
name chih_ming_lin
name wensheng_yang
name fosong_wang
name guorong_chen
name bo_ai
name yongjian_tang
name qi_hu
name ping_fu
name geng_yang
name li_zhu_wu
name da_xing
name jian_guo_he
name xiaobing_luo
name yang_shen
name lan_sun
name jian_du
name jianhua_lu
name jing_luo
name zhigang_zeng
name jian_feng
name hua_fu
name mei_xu
name ruijin_liao
name yu_ming_wang
name bo_hong
name weiming_zhu
name xu_xu
name rong_yu
name yong_tian
name lu_han
name lin_huang
name kexin_xu
name wei_quan
name tao_deng
name hongbin_li
name hua_bai
name mei_ling_chen
name yanqing_wang
name xu_dong_zhang
name qiang_shi
name min_zheng
name yue_tian
name jian_shao
name tian_chen
name yan_yan_li
nam

# 5. Local model_train stage

In [17]:
learning_rate = 0.01
epochs = 200
hidden1 = 128
hidden2 = 64
weight_decay = 0.0
dropout = 0.0
model = 'gcn_vae'
dataset_name = 'hui_fang'
is_sparse = 0
model_str = model 
name_str = dataset_name 

# Get current time
start_time = time.time()

In [18]:
# Using optimizers from tf.keras.optimizers
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

def compute_loss(preds, labels, model=None, num_nodes=None, pos_weight=1.0, norm=1.0):
    """
    Computes the loss function given predictions and labels.
    If model is not None and is a VAE model, the KL divergence loss is added.
    """
    cross_entropy_loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(logits=preds, labels=labels, pos_weight=pos_weight))
    cost = norm * cross_entropy_loss

    if model is not None and hasattr(model, 'z_mean') and hasattr(model, 'z_log_std'):
        kl_divergence = (0.5 / num_nodes) * tf.reduce_mean(tf.reduce_sum(1 + 2 * model.z_log_std - tf.square(model.z_mean) - tf.square(tf.exp(model.z_log_std)), 1))
        cost -= kl_divergence

    return cost

In [19]:
local_na_dir = join(settings.DATA_DIR, 'local', 'graph-{}'.format(IDF_THRESHOLD))

In [20]:
def extract_float_from_tensor_string(tensor_string):
    # Update regex to match floating point numbers more accurately
    match = re.search(r"\((-?\d+(\.\d+)?),", tensor_string)
    if match:
        return float(match.group(1))  # Convert the first matched group to float
    return 0.0  # If no match is found, returns 0.0

In [21]:
class Input_data:
    def __init__(self, idf_threshold):
        self.local_na_dir = join(settings.DATA_DIR, 'local', 'graph-{}'.format(idf_threshold))

    def encode_labels(self, labels):
        classes = set(labels)
        classes_dict = {c: i for i, c in enumerate(classes)}
        return list(map(lambda x: classes_dict[x], labels))

    def load_local_data(self, name='default_name'):
        # Load local paper network dataset
        print('Loading {} dataset...'.format(name), 'path=', self.local_na_dir)

        idx_features_labels = np.genfromtxt(join(self.local_na_dir, "{}_pubs_content.txt".format(name)), dtype=np.dtype(str))
        # Use helper functions to process each eigenvalue
        features = np.array([[extract_float_from_tensor_string(value) for value in row[1:-1]] for row in idx_features_labels], dtype=np.float32)
        labels = self.encode_labels(idx_features_labels[:, -1])

        # build graph
        idx = np.array(idx_features_labels[:, 0], dtype=str)  
        idx_map = {j: i for i, j in enumerate(idx)}
        edges_unordered = np.genfromtxt(join(self.local_na_dir, "{}_pubs_network.txt".format(name)), dtype=np.dtype(str))
        edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                         dtype=np.int32).reshape(edges_unordered.shape)
        adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                            shape=(features.shape[0], features.shape[0]), dtype=np.float32)

        # build symmetric adjacency matrix
        adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

        print('Dataset has {} nodes, {} edges, {} features.'.format(adj.shape[0], edges.shape[0], features.shape[1]))

        return adj, features, labels
        
idf_threshold = 10
input_data = Input_data(idf_threshold)

In [22]:
def weight_variable_glorot(input_dim, output_dim, name=""):
    """Create a weight variable with Glorot & Bengio (AISTATS 2010)
    initialization.
    """
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initial = tf.random_uniform([input_dim, output_dim], minval=-init_range,
                                maxval=init_range, dtype=tf.float32)
    return tf.Variable(initial, name=name)

In [23]:
# global unique layer ID dictionary for layer name assignment
_LAYER_UIDS = {}

In [24]:
def get_layer_uid(layer_name=''):
    """Helper function, assigns unique layer IDs
    """
    if layer_name not in _LAYER_UIDS:
        _LAYER_UIDS[layer_name] = 1
        return 1
    else:
        _LAYER_UIDS[layer_name] += 1
        return _LAYER_UIDS[layer_name]


def dropout_sparse(x, keep_prob, num_nonzero_elems):
    """Dropout for sparse tensors. Currently fails for very large sparse tensors (>1M elements)
    """
    noise_shape = [num_nonzero_elems]
    random_tensor = keep_prob
    random_tensor += tf.random_uniform(noise_shape)
    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
    pre_out = tf.sparse_retain(x, dropout_mask)
    return pre_out * (1. / keep_prob)


class Layer(tf.keras.layers.Layer):
    """Base layer class. Defines basic API for all layer objects.

    # Properties
        name: String, defines the variable scope of the layer.

    # Methods
        _call(inputs): Defines computation graph of layer
            (i.e. takes input, returns output)
        __call__(inputs): Wrapper for _call()
    """

    def __init__(self, **kwargs):
        super(Layer, self).__init__(**kwargs)
        self.issparse = False

    def _call(self, inputs):
        return inputs

    def __call__(self, inputs):
        with tf.name_scope(self.name):
            outputs = self._call(inputs)
            return outputs

class GraphConvolution(tf.keras.layers.Layer):
    def __init__(self, input_dim, output_dim, dropout_rate = 0., activation=tf.nn.relu, **kwargs):
        super(GraphConvolution, self).__init__(**kwargs)
        self.dropout_rate = dropout_rate
        self.activation = activation
        self.weight = self.add_weight(shape=(input_dim, output_dim), initializer='glorot_uniform', name='kernel')

    def call(self, inputs, training=False):
        x, adj_components = inputs
        indices = tf.cast(adj_components.indices, dtype=tf.int64)  # Get the indices of a sparse tensor
        values = adj_components.values  # Get the values of a sparse tensor
        dense_shape = adj_components.dense_shape  # Get the shape of a sparse tensor

        adj = tf.SparseTensor(indices=indices, values=values, dense_shape=dense_shape)  # Create a SparseTensor using the converted indices

        # print("Type of adj in GraphConvolution:", type(adj))  # Confirm the type of adj
        
        if training:
            x = tf.nn.dropout(x, rate=self.dropout_rate)
        
        x = tf.matmul(x, self.weight)
        x = tf.sparse.sparse_dense_matmul(adj, x)
        
        if self.activation is not None:
            x = self.activation(x)
        
        return x

class GraphConvolutionSparse(tf.keras.layers.Layer):
    def __init__(self, input_dim, output_dim, features_nonzero, dropout_rate=0., activation=tf.nn.relu, **kwargs):
        super(GraphConvolutionSparse, self).__init__(**kwargs)
        self.dropout_rate = dropout_rate
        self.activation = activation
        self.features_nonzero = features_nonzero
        self.weight = self.add_weight(shape=(input_dim, output_dim), initializer='glorot_uniform', name='kernel')

    def call(self, inputs, training=False):
        x, adj = inputs

        if training:
            x = dropout_sparse(x, 1 - self.dropout_rate, self.features_nonzero)
        
        x = tf.sparse.sparse_dense_matmul(x, self.weight)
        x = tf.sparse.sparse_dense_matmul(adj, x)
        
        if self.activation is not None:
            outputs = self.activation(x)
        return outputs

class InnerProductDecoder(tf.keras.layers.Layer):
    """Decoder model layer for link prediction."""

    def __init__(self, dropout_rate=0., activation=tf.nn.sigmoid, **kwargs):
        super(InnerProductDecoder, self).__init__(**kwargs)
        self.dropout_rate = dropout_rate
        self.activation = activation

    def call(self, inputs, training=False):
        if training and self.dropout_rate > 0:
            inputs = tf.nn.dropout(inputs, rate=self.dropout_rate)
        x = tf.transpose(inputs)
        x = tf.matmul(inputs, x)
        outputs = tf.reshape(x, [-1])
        return self.activation(outputs)


In [25]:
hidden1_units = 128
hidden2_units = 64

In [26]:
class Model(tf.keras.Model):
    def __init__(self, name=None, logging=False, **kwargs):
        super(Model, self).__init__(name=name, **kwargs)
        self.logging = logging
        self.issparse = False  # Set as needed to mark whether the input is sparse

    def get_embeddings(self, inputs):
        features, adj = inputs
        if hasattr(self, 'graph_conv1') and hasattr(self, 'graph_conv_mean'):
            x = features
            hidden1 = self.graph_conv1((x, adj), training=False)
            z_mean = self.graph_conv_mean((hidden1, adj), training=False)
            return z_mean
        else:
            raise NotImplementedError("The model does not implement the required layers for get_embeddings.")
            
class GCNModelAE(Model):
    def __init__(self, num_features, hidden1_units, hidden2_units, dropout_rate=0., name='GCNModelAE', **kwargs):
        super(GCNModelAE, self).__init__(name=name, **kwargs)
        self.dropout_rate = dropout_rate
        # Define model layer
        self.graph_conv1 = GraphConvolution(num_features, hidden1_units, activation=tf.nn.relu)
        self.graph_conv2 = GraphConvolution(hidden1_units, hidden2_units)
        self.inner_product_decoder = InnerProductDecoder()

    def call(self, inputs, training=False):
        features, adj = inputs
        x = features
        if training:
            x = tf.nn.dropout(x, rate=self.dropout_rate)
        hidden1 = self.graph_conv1((x, adj), training=training)
        embeddings = self.graph_conv2((hidden1, adj), training=training)
        reconstructions = self.inner_product_decoder(embeddings, training=training)
        return reconstructions

class GCNModelVAE(Model):
    def __init__(self, num_features, hidden1_units, hidden2_units, num_nodes, dropout_rate=0., name='GCNModelVAE', **kwargs):
        super(GCNModelVAE, self).__init__(name=name, **kwargs)
        self.dropout_rate = dropout_rate
        self.num_nodes = num_nodes
        # Define model layer
        self.graph_conv1 = GraphConvolution(input_dim=num_features, output_dim=hidden1_units, activation=tf.nn.relu)
        self.graph_conv_mean = GraphConvolution(input_dim=hidden1_units, output_dim=hidden2_units)
        self.graph_conv_log_std = GraphConvolution(input_dim=hidden1_units, output_dim=hidden2_units)
        self.inner_product_decoder = InnerProductDecoder()

    def call(self, inputs, training=False):
        features, adj = inputs
        x = features
        if training:
            x = tf.nn.dropout(x, rate=self.dropout_rate)
        hidden1 = self.graph_conv1([x, adj], training=training)
        z_mean = self.graph_conv_mean([hidden1, adj], training=training)
        z_log_std = self.graph_conv_log_std([hidden1, adj], training=training)
        z = z_mean + tf.random.normal([self.num_nodes, hidden2_units]) * tf.exp(z_log_std)
        reconstructions = self.inner_product_decoder(z, training=training)
        return reconstructions

model = Model()

In [27]:
class Preprocessing:
    def __init__(self):
        pass

    def sparse_to_tuple(self, sparse_mx):
        if not sp.isspmatrix_coo(sparse_mx):
            sparse_mx = sparse_mx.tocoo()
        coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
        values = sparse_mx.data
        shape = sparse_mx.shape
        return coords, values, shape

    def normalize_vectors(self, vectors):
        scaler = StandardScaler()
        vectors_norm = scaler.fit_transform(vectors)
        return vectors_norm

    def preprocess_graph(self, adj):
        adj = sp.coo_matrix(adj)
        adj_ = adj + sp.eye(adj.shape[0])
        rowsum = np.array(adj_.sum(1))
        degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())
        adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
        return self.sparse_to_tuple(adj_normalized)

    def gen_train_edges(self, adj):
        adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
        adj.eliminate_zeros()
        assert np.diag(adj.todense()).sum() == 0

        adj_triu = sp.triu(adj)
        adj_tuple = self.sparse_to_tuple(adj_triu)
        edges = adj_tuple[0]
        data = np.ones(edges.shape[0])
        adj_train = sp.csr_matrix((data, (edges[:, 0], edges[:, 1])), shape=adj.shape)
        adj_train = adj_train + adj_train.T
        return adj_train

    def cal_pos_weight(self, adj):
        pos_edges_num = adj.nnz
        return (adj.shape[0] * adj.shape[0] - pos_edges_num) / pos_edges_num

preprocessing = Preprocessing()

In [28]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np

def clustering(embeddings, num_clusters):
    # Check if there are any NaN values in the embeddings
    if np.isnan(embeddings).any():
        # Handle NaN values. Here, we simply remove them, but you could also impute them
        embeddings = embeddings[~np.isnan(embeddings).any(axis=1)]
        # If after removing NaN values there are fewer rows than num_clusters, adjust num_clusters
        num_clusters = min(num_clusters, len(embeddings))
    
    # Proceed with clustering if there are enough samples
    if len(embeddings) > 1:
        model = AgglomerativeClustering(n_clusters=num_clusters).fit(embeddings)
        return model.labels_
    else:
        # Handle the case where there are not enough samples left after removing NaN values
        print("Not enough samples left after removing NaN values for clustering")
        return None  # or handle this case as per your requirement


In [29]:
def gae_for_na(name):
    """
    train and evaluate disambiguation results for a specific name
    :param name: author name
    :return: evaluation results
    """
    
    adj, features, labels = input_data.load_local_data(name=name)

    adj_orig = adj - sp.diags([adj.diagonal()], [0])
    adj_orig.eliminate_zeros()
    adj_train = preprocessing.gen_train_edges(adj)
    adj_norm = preprocessing.preprocess_graph(adj_train)
    num_nodes = adj.shape[0]
    is_sparse = sp.issparse(features)  # is_sparse is defined here
    features = preprocessing.normalize_vectors(features) if not is_sparse else features.todense()

    features_tensor = tf.convert_to_tensor(features, dtype=tf.float32)
    adj_tensor = tf.SparseTensor(indices=adj_norm[0], values=adj_norm[1], dense_shape=adj_norm[2])

    if model_str == 'gcn_ae':
        model = GCNModelAE(num_features=features.shape[1], hidden1_units=128, hidden2_units=64, dropout_rate=0.5)
    elif model_str == 'gcn_vae':
        model = GCNModelVAE(num_features=features.shape[1], hidden1_units=128, hidden2_units=64, num_nodes=num_nodes, dropout_rate=0.5)
    else:
        raise ValueError("Invalid model_str")
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    for epoch in range(epochs):
        t = time.time()
        with tf.GradientTape() as tape:
            reconstructions = model([features_tensor, adj_tensor], training=True)
            adj_orig_dense = tf.sparse.to_dense(tf.sparse.reorder(tf.SparseTensor(indices=np.array(adj_orig.nonzero()).T, values=adj_orig.data, dense_shape=adj_orig.shape)))
            loss = compute_loss(tf.reshape(reconstructions, adj_orig_dense.shape), adj_orig_dense)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        print(f"Epoch: {epoch+1}, train_loss: {loss.numpy():.5f}, time: {time.time() - t:.5f}")

    # Get the embed and process it later
    emb = model.get_embeddings((features_tensor, adj_tensor))
    n_clusters = len(set(labels))
    emb_norm = preprocessing.normalize_vectors(emb)
    clusters_pred = clustering(emb_norm, num_clusters=n_clusters)

    # Check if clusters_pred is None before calling pairwise_precision_recall_f1
    if clusters_pred is not None:
        prec, rec, f1 = eval_utils.pairwise_precision_recall_f1(clusters_pred, labels)
    else:
        # If clusters_pred is None, sets the evaluation metric to its default value
        prec, rec, f1 = 0, 0, 0
        print('Warning: Since clusters_pred is None, precision, recall, and F1 score are set to 0.')

    print('pairwise precision', '{:.5f}'.format(prec), 'recall', '{:.5f}'.format(rec), 'f1', '{:.5f}'.format(f1))
    return [prec, rec, f1], num_nodes, n_clusters

def load_test_names():
    return data_utils.load_json(settings.DATA_DIR, 'test_name_list.json')

In [ ]:
names = load_test_names()
wf = codecs.open(join(settings.OUT_DIR, 'local_clustering_results.csv'), 'w', encoding='utf-8')
wf.write('name,n_pubs,n_clusters,precision,recall,f1\n')
metrics = np.zeros(3)
cnt = 0
for name in names:
    cur_metric, num_nodes, n_clusters = gae_for_na(name)
    wf.write('{0},{1},{2},{3:.5f},{4:.5f},{5:.5f}\n'.format(
        name, num_nodes, n_clusters, cur_metric[0], cur_metric[1], cur_metric[2]))
    wf.flush()
    for i, m in enumerate(cur_metric):
        metrics[i] += m
    cnt += 1
    macro_prec = metrics[0] / cnt
    macro_rec = metrics[1] / cnt
    macro_f1 = eval_utils.cal_f1(macro_prec, macro_rec)
    print('average until now', [macro_prec, macro_rec, macro_f1])
    time_acc = time.time()-start_time
    print(cnt, 'names', time_acc, 'avg time', time_acc/cnt)
macro_prec = metrics[0] / cnt
macro_rec = metrics[1] / cnt
macro_f1 = eval_utils.cal_f1(macro_prec, macro_rec)
wf.write('average,,,{0:.5f},{1:.5f},{2:.5f}\n'.format(
    macro_prec, macro_rec, macro_f1))
wf.close()

Loading hongbin_liang dataset... path= /home/netdb/project/Name_Disambiguation_BERT/data/local/graph-10
Dataset has 183 nodes, 6072 edges, 192 features.
Epoch: 1, train_loss: 0.81994, time: 0.13154
Epoch: 2, train_loss: 0.81368, time: 0.01521
Epoch: 3, train_loss: 0.82009, time: 0.01390
Epoch: 4, train_loss: 0.81639, time: 0.01369
Epoch: 5, train_loss: 0.82292, time: 0.01344
Epoch: 6, train_loss: 0.81985, time: 0.01350
Epoch: 7, train_loss: 0.81800, time: 0.01351
Epoch: 8, train_loss: 0.81865, time: 0.01302
Epoch: 9, train_loss: 0.81894, time: 0.01321
Epoch: 10, train_loss: 0.82019, time: 0.01368
Epoch: 11, train_loss: 0.81856, time: 0.01330
Epoch: 12, train_loss: 0.82143, time: 0.01336
Epoch: 13, train_loss: 0.81891, time: 0.01353
Epoch: 14, train_loss: 0.82093, time: 0.01359
Epoch: 15, train_loss: 0.81888, time: 0.01378
Epoch: 16, train_loss: 0.81624, time: 0.01380
Epoch: 17, train_loss: 0.81790, time: 0.01293
Epoch: 18, train_loss: 0.81946, time: 0.01355
Epoch: 19, train_loss: 0.819

/home/netdb/anaconda3/lib/python3.11/site-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/netdb/anaconda3/lib/python3.11/site-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/home/netdb/anaconda3/lib/python3.11/site-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Not enough samples left after removing NaN values for clustering
警告: 由于 clusters_pred 为 None，精确度、召回率和 F1 分数被设置为 0。
pairwise precision 0.00000 recall 0.00000 f1 0.00000
average until now [0.4778996776054531, 0.8137456869542565, 0.6021603330428548]
31 names 184.09586358070374 avg time 5.93857624453883
Loading yu_ming_wang dataset... path= /home/netdb/project/Name_Disambiguation_BERT/data/local/graph-10
Dataset has 227 nodes, 1649 edges, 192 features.
Epoch: 1, train_loss: 0.97226, time: 0.11048
Epoch: 2, train_loss: 0.96663, time: 0.01314
Epoch: 3, train_loss: 0.97059, time: 0.01118
Epoch: 4, train_loss: 0.97090, time: 0.01136
Epoch: 5, train_loss: 0.96791, time: 0.01140
Epoch: 6, train_loss: 0.96856, time: 0.01121
Epoch: 7, train_loss: 0.96928, time: 0.01137
Epoch: 8, train_loss: 0.96957, time: 0.01104
Epoch: 9, train_loss: 0.96666, time: 0.01103
Epoch: 10, train_loss: 0.96609, time: 0.01097
Epoch: 11, train_loss: 0.97209, time: 0.01116
Epoch: 12, train_loss: 0.97116, time: 0.01103
Epoc